import: Es el comando que le dice a Python "Ve a buscar esta herramienta".

pandas: Es el nombre de la librería que queremos traer.

as pd: Esto es un alias. En lugar de escribir pandas cada vez que queramos usarla, le ponemos un apodo corto: pd. Es un estándar en la industria; todo el mundo lo llama pd

In [20]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from google.colab import files:

Concepto: Estamos importando un módulo (una pequeña herramienta especializada) que Google creó para manejar archivos en su entorno. Es como abrir una ventana de comunicación directa.

files.upload():

Concepto: Esta función abre una pequeña ventana en tu navegador o VS Code que te permite buscar y seleccionar tu archivo local (all-states-history.csv). Una vez seleccionado, ¡el archivo se copia al servidor remoto!

In [19]:
#from google.colab import drive
#drive.mount('/content/drive')

Cargar Archivo CSV, tambien podemos leer archivos JSON, XML

In [18]:
# df = pd.read_csv("all-states-history.csv")
# df.head()

Inspección de la Estructura



df.shape (La Forma):

Concepto: Piensa en esto como preguntar: "¿De qué tamaño es nuestra tabla?". Devuelve una tupla (filas, columnas). El primer número son las observaciones (filas o días/estados en este caso) y el segundo número son las variables (columnas o características).

df.info() (El Reporte de Calidad):

Concepto: Este es nuestro reporte más importante. Nos dice, columna por columna, tres cosas fundamentales:

Tipo de dato (Dtype): ¿Python lo está interpretando como texto (object), número entero (int64), número decimal (float64) o fecha? Esto es crucial.

Valores No Nulos (Non-Null Count): ¿Cuántas celdas tienen datos en esa columna? Si este número es menor que el número total de filas, significa que hay valores perdidos o valores nulos (NaN).

In [ ]:
# Muestra el tamaño del DataFrame (filas, columnas)
print(df.shape) 

# Muestra un resumen de cada columna: nombre, tipo de dato y valores no nulos.
df.info()

Observa la salida de df.info(). Te enfocarás en dos columnas que son clave en este conjunto de datos (que es de seguimiento de COVID-19):

La columna date (fecha) 🗓️.

Las columnas que representan conteos como death, hospitalized, positive (casos, muertes, hospitalizados) 🔢.

La columna Date, la cambiaremos a datetime

Concepto Clave: Tipos de Datos 🏷️ Si la fecha se queda como object (texto), Pandas no la entiende como un punto en el tiempo. Sería como si Python viera "2021-03-07" simplemente como una secuencia de caracteres, no como algo que puede ordenar o sobre lo que puede calcular una tendencia. Para hacer un buen Análisis Exploratorio de Datos (EDA) y cumplir con la evaluación, la conversión es obligatoria.

Conversión de Tipo de Dato

Para solucionar esto, usamos la función pd.to_datetime() de Pandas.


df['date']: Estamos seleccionando la columna date de nuestro DataFrame. Piensa en esto como si le pusieras el dedo encima a esa columna específica en tu tabla.

pd.to_datetime(...): Es la receta mágica de Pandas que toma la columna de texto y la transforma en un formato de fecha reconocido.

df['date'] = ...: Esto es una reasignación. Tomamos la columna recién convertida y la guardamos de nuevo en el mismo lugar, sobrescribiendo la columna de texto original.

In [ ]:
df["date"] = pd.to_datetime(df["date"])

In [ ]:
df.info()

Contar los Valores Faltantes
Aunque df.info() nos da una idea (las cuentas que no llegan al total de filas), queremos ver el número exacto, ordenado por columna.

Vamos a usar una combinación de comandos muy útil en Pandas

Ahora, pasemos a la Limpieza de Datos, específicamente la Detección y Tratamiento de Valores Faltantes (o Valores Nulos, denotados como NaN en Pandas)

df.isnull():

Le pregunta a Pandas, celda por celda: "¿Es esta celda nula?". Devuelve una tabla idéntica a df pero llena solo de True (si es nulo) o False (si no lo es).

.sum():

Como Python trata True como el número 1 y False como 0, al aplicar .sum() al resultado de isnull(), suma todos los Trues, dándonos el conteo total de valores nulos por cada columna.

In [ ]:
# Cuenta el número de valores nulos por cada columna
df.isnull().sum()

Tu conteo nos dice, por ejemplo:

deathConfirmed tiene 11,358 valores faltantes.

deathProbable tiene 13,187 valores faltantes.

Algunas como onVentilatorCumulative tienen hasta 19,490 faltantes.

Ahora viene la parte crítica del Preprocesado: decidir qué hacer con esos huecos.

La Regla del Porcentaje
El DataFrame tiene un total de 20,780 filas. Calcular el porcentaje de valores faltantes es clave.



| **Columna** | **Porcentaje faltante** |
|---|---:|
| `positiveTestsPeopleAntigen` | ≈ **97%** |
| `negativeTestsPeopleAntibody` | ≈ **95%** |
| `onVentilatorCumulative` | ≈ **94%** |
| `inIcuCumulative` | ≈ **82%** |


Concepto Clave: Imputación vs. Eliminación 🗑️Imputar (rellenar) funciona si falta un 5% o un 10% de los datos, usando la media o la mediana.Si falta un $\mathbf{80\%}$ o $\mathbf{90\%}$, ¡no podemos rellenar! Estaríamos inventando la inmensa mayoría de los datos.La decisión de analista aquí es clara: debemos eliminar las columnas que están mayormente vacías, ya que no aportan información fiable.

Librerias para graficas, visualizaciones de datos faltantes

import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

In [ ]:
# Cuenta nulos y los grafica
df.isna().sum().plot(kind='bar', figsize=(10, 5), color='skyblue')
plt.title("Cantidad de Nulos por Columna")
plt.ylabel("Número de valores faltantes")
plt.show()

In [ ]:
# Calcula la media, multiplica por 100 y añade el símbolo %
porcentaje_nulos = (df.isna().mean() * 100).map("{:.2f}%".format)
print(porcentaje_nulos)


| **Columna** | **Porcentaje faltante** |
|---|---:|
| `date` | 0.00% |
| `state` | 0.00% |
| `death` | 4.09% |
| `deathConfirmed` | 54.66% |
| `deathIncrease` | 0.00% |
| `deathProbable` | 63.46% |
| `hospitalized` | 40.41% |
| `hospitalizedCumulative` | 40.41% |
| `hospitalizedCurrently` | 16.56% |
| `hospitalizedIncrease` | 0.00% |
| `inIcuCumulative` | 81.77% |
| `inIcuCurrently` | 44.00% |
| `negative` | 36.04% |
| `negativeIncrease` | 0.00% |
| `negativeTestsAntibody` | 92.98% |
| `negativeTestsPeopleAntibody` | 95.32% |
| `negativeTestsViral` | 75.82% |
| `onVentilatorCumulative` | 93.79% |
| `onVentilatorCurrently` | 56.08% |
| `positive` | 0.90% |
| `positiveCasesViral` | 31.44% |
| `positiveIncrease` | 0.00% |
| `positiveScore` | 0.00% |
| `positiveTestsAntibody` | 83.90% |
| `positiveTestsAntigen` | 89.25% |
| `positiveTestsPeopleAntibody` | 94.74% |
| `positiveTestsPeopleAntigen` | 96.95% |
| `positiveTestsViral` | 56.89% |
| `recovered` | 42.24% |
| `totalTestEncountersViral` | 74.83% |
| `totalTestEncountersViralIncrease` | 0.00% |
| `totalTestResults` | 0.80% |
| `totalTestResultsIncrease` | 0.00% |
| `totalTestsAntibody` | 76.95% |
| `totalTestsAntigen` | 83.54% |
| `totalTestsPeopleAntibody` | 89.41% |
| `totalTestsPeopleAntigen` | 95.19% |
| `totalTestsPeopleViral` | 55.74% |
| `totalTestsPeopleViralIncrease` | 0.00% |
| `totalTestsViral` | 30.14% |
| `totalTestsViralIncrease` | 0.00% |

> ⚠️ **Nota:** Columnas con mas de 90%, 80, 70, 60, seran eliminadas

In [ ]:
# 1. Calcula los porcentajes de nulos
porcentajes_nulos = df.isnull().mean() * 100

# 2. Filtra los nombres de las columnas que superan el 60%
umbral_minimo = 60.0 
columnas_a_eliminar = porcentajes_nulos[porcentajes_nulos > umbral_minimo].index.tolist()

print("Columnas que superan el 60% de nulos:")
print(columnas_a_eliminar)

In [ ]:
# Eliminamos las columnas utilizando la lista automatizada.
# El argumento 'columns' y 'inplace=True' son la clave.
df.drop(columns=columnas_a_eliminar, inplace=True)